In [1]:
# 2020-12-20
# Batch para espectros do TP e RF de 2019
# Composição e síntese dos códigos: 
# - openGamma_pkl-generation: coleta dos arquivos
# - openGamma_verbose.ipynb : análise dos espectros

In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Feb 22 10:40:56 2020

@author: marcelo

ar1:
- array (nx,2), onde:
    nx: n'umero total de picos em todas as escalas
    x: 'indice do picoh
    ar1[x,0]: canal inteiro do pico
    ar1[x,1]: valor da escala
    
xpk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise
    x: canal inteiro do pico
    
ypk:
- lista (nx), onde:
    nx: numero de picos efetivos ap'os an'alise (= nx(xpk))
    x: valor da escala

"""

import numpy as np
import numpy.ma as ma  # masked array
import scipy.stats as sta # statistics
from scipy.special import (erf, erfinv)

from numpy import linalg as LA

from spectrum_analysis import (cwt_complete_analysis, gaussian_complete_analysis, cwt_calc_peaks)
from gamma_spectrum_class import (Spec)
####################################
# Para a coleta dos espectros:
import time
from machine_selection import set_screen_size_by_machine
from filebatch_class import (FileBatch)
from pathlib import (Path)
####################################


from bokeh.models import ColumnDataSource, Whisker, Label, LabelSet, MultiLine
from bokeh.palettes import Paired6
from bokeh.plotting import figure, output_file, output_notebook, show
from bokeh.layouts import gridplot, column, row

from bokeh import palettes

# 2020-08-22 Para rodar curve_fit
from gauss_funcs import func_example, func_example_fixed_b
# 2020-09-03 Desativei por enquanto
# import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
# 2020-08-30
from scipy.interpolate import splrep, splev

from scipy.ndimage import label, generate_binary_structure, find_objects, binary_dilation  # 2020-09-03
from numpy.polynomial import Polynomial as P # 2020-09-06 Esta é a nova classe recomendada 

# 2020-09-11
from opengamma_searchfuncs import selectOverlappedPks

# 2019-03-15
# https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html
import pandas as pd
import csv


In [3]:

# Para espectros CTP 4k
kSigma = 4.0 # Com 3.0 ele pega os grandes degraus Compton, com 4.0 só o da esquerda (retroespalhamento)
aSta = 0.004
bSta = 3.0
kExpand = 1 # Parâmetro para alargar região (inteiro positivo)
# 2020-12-05 Ainda não aprendi a mexer com o smoo... Chutando bem baixo para 4k, altíssima intens
smoo = 0.1 # Tem que ser >= 0.0
yrange = [10**1, 10**6]
# fn2 = '../Ipen/Efluentes-gasosos/Filtros-espectros/2018/CTP/CTP2203-3.Chn'


In [4]:
fb = FileBatch()
pathPosixName =  '../Ipen/Efluentes-gasosos/Filtros-espectros/2019'
fb.slotSetBatchCHN( pathPosixName )
print (fb.numarqstxt, ' arquivos')

../Ipen/Efluentes-gasosos/Filtros-espectros/2019
Num de arquivos:        374 


  arquivos


In [5]:
lp_posix = [Path(fn).as_posix() for fn in fb.lp]

In [6]:
all_polys_specs = []
all_specs_names = []
all_specs_LTs = []
all_specs_start_cnt = []
for fn2 in lp_posix:
    spec2 = Spec()
    if spec2.readSp( fn2 ) >= 0:
        print (fn2)
        all_specs_names.append(fn2)
        all_specs_LTs.append(str(spec2.spLVTime))
        all_specs_start_cnt.append(str(spec2.countDatetime))
        counts = spec2.specIO.spCounts
        print(len(counts))
        if (spec2.chancalib.size >= 3) & (spec2.enChcalib.size == spec2.chancalib.size):
            spec2.coeffs_ChEn = P.fit(spec2.chancalib, spec2.enChcalib, deg=2)
        if (spec2.enFwcalib.size >= 3) & (spec2.fwhmcalib.size == spec2.enFwcalib.size):
            spec2.coeffs_EnFw = P.fit(spec2.enFwcalib, spec2.fwhmcalib, deg=2)
        xs = spec2.xs
        y0s = spec2.y0sCorr
        lnCounts = spec2.lnCounts
        nCh = xs.size
        # Calcular espectro com ajuste parabólico na mesma janela e calculado no centro da janela
        parabSpec = np.zeros(nCh)
        hWIni = spec2.hStaW(0,a=aSta,b=bSta)
        hWFin = spec2.hStaW(nCh-1,a=aSta,b=bSta)
        for i in range(hWIni,nCh-hWFin):
            hW = spec2.hStaW(i,a=aSta,b=bSta)
            chanWin = np.array(range(i-hW,i+hW+1))
            lnctsWindow = lnCounts[chanWin]
            # Primeiro, acha usando parábola na janela e calcula no canal central
            parabWinFit = P.fit(chanWin,lnctsWindow,deg=2)
            parabSpec[i]=parabWinFit(i)
        tckDirectSpl = splrep(xs, parabSpec, k=3, s=smoo)
        smooParabSpec = np.exp(splev(xs, tckDirectSpl, der=0))
        del parabSpec
        isRegionSigma = np.abs(y0s-smooParabSpec) / (np.sqrt(y0s+5))
        isReg = isRegionSigma > kSigma
        # 2020-12-06
        # Expandindo canais marcados para chExpand = kExpand * hStaW(ch) canais antes e depois
        # Deu certo: depois, eliminar chExpand fixo.
        isRegExpand = np.zeros(nCh)
        chExpandIni = kExpand*spec2.hStaW(0,a=aSta,b=bSta)
        chExpandFin = kExpand*spec2.hStaW(nCh-1,a=aSta,b=bSta)
        for i in range(chExpandIni, nCh-chExpandFin+1):
            chExpandMov = kExpand*spec2.hStaW(i,a=aSta,b=bSta)
            isRegExpand[i] = np.any(isReg[i-chExpandMov:i+chExpandMov+1])
        # Com base nos canais marcados, definir extremos das regiões
        comuta = np.zeros(nCh)
        for i in range(1,nCh):
            comuta[i] = isRegExpand[i] - isRegExpand[i-1]
        # inis = np.insert(np.nonzero(comuta>0), 0, 0)
        # np.nonzero gera uma tupla, não sei por quê.
        inis = np.nonzero(comuta>0)[0]
        # fins = np.append(np.nonzero(comuta<0), n)
        fins = np.nonzero(comuta<0)[0]
        # Ajusta comprimento dos arrays. Têm de ser iguais.
        minSize = np.minimum(inis.size, fins.size)
        inis = inis[:minSize] 
        fins = fins[:minSize]
        mix = np.concatenate(np.array([[inis], [fins]])).T
        # Agora, criar nova baseline onde
        # Fora das regiões marcadas, seja smooParabSpec
        # Nas regiões marcadas, seja a stepLine dos extremos
        assembledBaseLine = np.array(smooParabSpec)
        for regLim in mix:
            assembledBaseLine[regLim[0]:regLim[1]+1] = spec2.stepBaseLine (
                regLim[0], regLim[1], smooParabSpec[regLim[0]], smooParabSpec[regLim[1]], y0s)
        lnNetCounts = np.zeros(nCh)
        wLnNetCounts = np.zeros(nCh)
        uncys = np.sqrt(y0s)
        contin = assembledBaseLine
        yNets = y0s - contin
        for i in range(nCh):
            if yNets[i] < 2:
                lnNetCounts[i] = 0
                wLnNetCounts[i] = 0
            else:
                lnNetCounts[i] = np.log(yNets[i])
                wLnNetCounts[i] = np.sqrt(y0s[i])
        # 2020-12-16
        # Usando as regiões mix, ir direto ao ajuste parabólico
        # Quebrar yNets usando mix
        # Testar se é uma gaussiana simples ou multipleto
        # 2020-12-16  Agora vou usar split para dividir o array. Vejamos:
        # 2020-12-18 AQUI: 1e-3 picota demais. Usar:
        cutFactor = 1e-5
        # Ok. Agora, montando tudo: e acrescentando os canais xs correpondentes, e também uncys
        xsUsefulSubRegions = []
        usefulSubRegions = []
        wsUsefulSubRegions = []
        for regLim in mix:
            # print('===============')
            xsNetReg = xs[regLim[0]:regLim[1]]
            netRegion = yNets[regLim[0]:regLim[1]]
            #  wsNetReg = uncys[regLim[0]:regLim[1]]  # Nao bom com uncys. Tentar com:
            netRegion[netRegion < 0.0] = 0.0
            wsNetReg = netRegion
            maxCountInReg = np.max(netRegion)
            isToSplit = (netRegion / maxCountInReg) < cutFactor
            comuta = np.zeros(isToSplit.size)
            for i in range(1,isToSplit.size):
                comuta[i] = isToSplit[i] ^ isToSplit[i-1]
            splitting, = comuta.nonzero() # Precisa da vírgula pois é tupla
            xsSplitted = np.split(xsNetReg,splitting)
            splitted = np.split(netRegion,splitting)
            wsSplitted = np.split(wsNetReg,splitting)
            # print(splitted)
            # print('-----')
            # for i in range(0,len(splitted),2):
            #     print(splitted[i])
            # for j in range(1,len(splitted),2):
            #     print(splitted[j])
            if not isToSplit[0]:
                xsUsefulSubRegions.append(xsSplitted[0])
                usefulSubRegions.append(splitted[0])
                wsUsefulSubRegions.append(wsSplitted[0])
            for i in range(splitting.size):
                if not isToSplit[splitting[i]]:
                    xsUsefulSubRegions.append(xsSplitted[i+1])
                    usefulSubRegions.append(splitted[i+1])
                    wsUsefulSubRegions.append(wsSplitted[i+1])
        newregsMarks = [np.ones_like(x)*1.0e4 for x in xsUsefulSubRegions]
        xspolys = []
        polys = []
        for ixs, xsreg in enumerate(xsUsefulSubRegions):
            ysreg=usefulSubRegions[ixs]
            wsreg=wsUsefulSubRegions[ixs]
            if xsreg.size > 2:
                xspolys.append(xsreg)
                polys.append(P.fit(xsreg,np.log(ysreg),w=wsreg,deg=2))
        # 2020-12-18
        # PAREI AQUI: Agora, partir direto para o cálculo dos picos a partir de polys
        # Está tudo na list polys
        # 2020-12-20 Depois de calcular polys, montar o np.array dos picos e
        # Salvar num csv usando numpy.savetxt
        all_polys_specs.append(polys)

../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Ctp/CTP0205-2.Chn
4096
../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Ctp/CTP2401-3.Chn
4096
../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Ctp/CTP0702-1.Chn
4096
../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Ctp/CTP1212-2 (7,31%).Chn
4096
../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Ctp/CTP2811-2 (7,31%).Chn
4096
../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Ctp/CTP0507.Chn
4096
../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Ctp/CTP2609-2 (9,26%).Chn
4096
../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Ctp/CTP11073.Chn
4096
../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Ctp/CTP1001-2.Chn
4096
../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Ctp/CTP1807.Chn
4096
../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Ctp/CTP3005-3 (7,86%).Chn
4096
../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Ctp/CTP0310-2 (10,05%).Chn
4096
../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Ctp/CTP2401-5.Chn
4096
../Ipen/Efluentes-gasosos/Fi

In [8]:
numSpe = len(all_specs_names)
print(numSpe)
results_areas = np.empty([numSpe,13],dtype=object)
results_areas

374


array([[None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       ...,
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None],
       [None, None, None, ..., None, None, None]], dtype=object)

In [14]:
all_spec_pEns = []
spec2 = Spec()
for fn2 in all_specs_names:
    if spec2.readSp( fn2 ) >= 0:
        all_spec_pEns.append(spec2.pEn)
# Tem que ter o mesmo len de all_specs_names!
print(len(all_spec_pEns))

374


In [113]:
# Constante para cálculo da área do pico
karea = 0.5*np.sqrt(np.pi/np.log(2.0))

for ipEn, pols_in_a_spec in enumerate(all_polys_specs):
    polys_conv = []
    for pol in pols_in_a_spec:
        polconv = pol.convert().coef
        if polconv[2] < 0:
            polys_conv.append(polconv)
    polys_conv_ar = np.array(polys_conv)
    linha = np.empty(13,dtype=object)
    linha[0]=all_specs_names[ipEn]
    linha[1]=all_specs_LTs[ipEn].replace('.',',')
    linha[2]=all_specs_start_cnt[ipEn]
    if polys_conv_ar.shape[0]!=0:
        a0 = polys_conv_ar[:,0]
        a1 = polys_conv_ar[:,1]
        a2 = polys_conv_ar[:,2]
        sigmas = 1.0/(np.sqrt(2*(-a2)))
        fwhms = 2*np.sqrt(2*np.log(2))*sigmas
        chCentroids = -a1/(2*a2)
        alturas = np.exp(a0 - a1**2 / (4*a2))
        areas = karea * alturas * fwhms
        max5 = np.argsort(areas)[-5:]
        maxareas = np.array(areas[max5])
        enCtroids = all_spec_pEns[ipEn](chCentroids)
        maxens = enCtroids[max5]
        # AQUI para posicionar direito em linha:
        nmaxar = maxareas.shape[0]
        str1 = np.array([str(x).replace('.',',') for x in maxareas])
        str2 = np.array([str(x).replace('.',',') for x in maxens])
        # Tem que ser 3 para nmaxar==5, 4 para nmaxar==4, etc.
        linha[8-nmaxar:8] = str1
        linha[13-nmaxar:13] = str2
    results_areas[ipEn] = linha

In [114]:
print(karea)

1.0644670194312262


In [115]:
results_areas.shape

(374, 13)

In [116]:
results_areas

array([['../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Ctp/CTP0205-2.Chn',
        '5000,0', '2019-05-17 14:31:23', ..., '632,7443174308075',
        '286,37681332407146', '365,05817327322427'],
       ['../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Ctp/CTP2401-3.Chn',
        '5000,0', '2019-02-06 17:24:48', ..., '646,2507536593064',
        '293,4540367261361', '373,56778043857594'],
       ['../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Ctp/CTP0702-1.Chn',
        '5000,0', '2019-02-18 14:45:12', ..., '623,9586766373185',
        '271,1199689595643', '351,2626869549523'],
       ...,
       ['../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Prf/PRF1005.Chn',
        '5000,0', '2019-05-10 14:36:14', ..., None, None,
        '-18,454522654393703'],
       ['../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Prf/PRF1508.Chn',
        '5000,0', '2019-08-15 15:00:36', ..., None, None, None],
       ['../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Prf/PRF0606.Chn',
        '5000,

In [117]:
# 2020-12-22 E finalmente...

In [118]:
np.savetxt('2019_TP_RF_results.csv', results_areas, fmt='%s', delimiter=';', newline='\n')

In [119]:
results_areas[-1]

array(['../Ipen/Efluentes-gasosos/Filtros-espectros/2019/Prf/PRF0606.Chn',
       '5000,0', '2019-06-06 14:00:32', None, '3,375161783507002',
       '7,238284290412698', '270,71087367302863', '430,80286593822893',
       None, '0,8193462853564809', '597,2784476468258',
       '592,478357411088', '255,07429868631363'], dtype=object)

In [ ]:
### 2020-12-20 FIM!